In [53]:
import torch
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import PCAM
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm as _tqdm

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [50]:
transform = transforms.Compose([
    transforms.PILToTensor()
])

In [51]:
train_dataset = PCAM(root='data', split='test', download=True, transform=transform)
val_dataset = PCAM(root='data', split='test', download=True, transform=transform)
# The dataset needs to be fixed so that we have a list of tensors not PIL images

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)


In [52]:
model = resnet18(pretrained=True)

# Freeze all layers except last
for param in model.parameters():
    param.requires_grad = False
    
num_classes = 2
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)


In [54]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    
    model.to(device)
    
    # Train the model for the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the train loader
        for inputs, labels in tqdm(train_loader):
            # Move the inputs and labels to the device
            inputs = inputs.float().to(device)
            labels = labels.to(device)

            # Zero the optimizer gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy
        train_loss = running_loss / len(train_dataset)
        train_acc = running_corrects.double() / len(train_dataset)

        # Set the model to evaluation mode
        model.eval()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the validation loader
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                # Move the inputs and labels to the device
                inputs = inputs.float().to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Update the running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        # Calculate the validation loss and accuracy
        val_loss = running_loss / len(val_dataset)
        val_acc = running_corrects.double() / len(val_dataset)

        # Print the epoch results
        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}, val loss: {:.4f}, val acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))

In [ ]:
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, device=device)

100%|██████████| 1024/1024 [00:29<00:00, 34.96it/s]


Epoch [1/5], train loss: 0.8886, train acc: 0.7557, val loss: 0.7043, val acc: 0.7968


100%|██████████| 1024/1024 [00:29<00:00, 34.52it/s]


Epoch [2/5], train loss: 0.9222, train acc: 0.7609, val loss: 0.9153, val acc: 0.7637


100%|██████████| 1024/1024 [00:29<00:00, 34.62it/s]


Epoch [3/5], train loss: 1.0040, train acc: 0.7643, val loss: 0.9202, val acc: 0.7988


100%|██████████| 1024/1024 [00:30<00:00, 33.60it/s]


Epoch [4/5], train loss: 0.9378, train acc: 0.7641, val loss: 0.6289, val acc: 0.8063


  3%|▎         | 33/1024 [00:01<00:30, 32.91it/s]